##### --- Day 12: Garden Groups ---

--- Part One ---

In [34]:
from itertools import groupby

file = open("day_12.txt", "r")
garden = [[i for i in j] for j in [i.strip() for i in file.readlines()]]

In [47]:
def in_garden(pos, garden):
    if pos[0] > len(garden) - 1:
        return False
    if pos[1] > len(garden[0]) - 1:
        return False
    if pos[0] < 0 or pos[1] < 0:
        return False
    return True

def free_sides(pos, letter, garden):
    free_sides = 0
    x = pos[0]
    y = pos[1]
    compass = [[x-1, y ], [x+1, y], [x, y-1], [x, y+1]]
    # Check up, down, left right for adjacent letters.
    for p in compass:
        if not in_garden(p, garden):
            free_sides += 1
        elif garden[p[0]][p[1]] != letter:
            free_sides += 1
        else:
            pass
    return free_sides

In [48]:
def recursive_crawler(positions, garden, letter, discovered):
    if discovered == 0:
        return positions
    else:
        new_discoveries = 0
        # For each discovered new area, expand outwards and see if NSEW also in area.
        for pos in positions:
            x = pos[0]
            y = pos[1]
            compass = [[x-1, y ], [x+1, y], [x, y-1], [x, y+1]]
            
            for p in compass:
                # Check up, down, left right for matching letters.
                x = p[0]
                y = p[1]
                if not in_garden(p, garden):
                    pass
                elif garden[p[0]][p[1]] == letter:
                    if p not in positions:  
                        positions.append(p)
                        new_discoveries += 1
                else:
                    pass
        # Work recursively through all points to find new points.
        return recursive_crawler(positions, garden, letter, new_discoveries)

area_areas = []
total = 0
area_id = 0
areas = {}
all_positions = []
for r_ind, row in enumerate(garden):
    for c_ind, l in enumerate(row):
        if [r_ind, c_ind] not in all_positions:
            # Find all positions in that area.
            a_pos = recursive_crawler([[r_ind, c_ind]], garden, l, 1)

            # Add all positions to the all_positions list.
            all_positions += a_pos

            # Add the area multiplied by the perimeter to the total cost.
            total += len(a_pos) * sum(free_sides(p, l, garden) for p in a_pos) 
            area_areas.append(len(a_pos))
            
            # Add all edge letters to areas dictionary and increment area_id.
            areas[(area_id, l)] = a_pos
            area_id += 1

print(total)

1461806


--- Part Two ---

In [10]:
def sides_facing(pos, garden):
    # Returns a list of compass points that the free sides are facing in ["N", "S", "E"] format.
    sides_facing = []
    x = pos[0]
    y = pos[1]
    area_letter = garden[x][y]
    compass = {"N":[x-1, y ], "S":[x+1, y], "E": [x, y+1], "W":[x, y-1]}
    # Check up, down, left right for absense of adjacent letters.
    for p in compass:
        if not in_garden(compass[p], garden):
            sides_facing.append(p)
        elif garden[compass[p][0]][compass[p][1]] != area_letter:
            sides_facing.append(p)
        else:
            pass
    return sides_facing

In [45]:
new_total = 0
area_counter = 0
for a in areas:
    facing = {"N": [], "S": [], "E": [], "W": []}
    for p in areas[a]:
        # Find which directions each free side is facing and add to facing dictionary for each one.
        p_facing = sides_facing(p, garden)
        for d in p_facing:
            facing[d].append(p)

    area_sides = 0
    for c_p in facing:
        # For each direction strip out adjacent letters.
        if c_p in "NS":
            facing[c_p] = sorted(facing[c_p])
            sorted_rows = [list(group) for key, group in groupby(facing[c_p], lambda x: x[0])]
            for row in sorted_rows:
                to_remove = []
                current = None
                for l in row:
                    if current == None:
                        current = l
                    elif l[1] == current[1] + 1:
                        # Must be adjacent, add to the to remove list.
                        to_remove.append(current)
                        current = l
                    else:
                        # Not at the start, and not adjacent, so is a break in the side.
                        current = l
                area_sides += len([x for x in row if x not in to_remove])
        else:
            facing[c_p] = sorted(facing[c_p], key=lambda x: x[1])  
            sorted_rows = [list(group) for key, group in groupby(facing[c_p], lambda x: x[1])]
            for row in sorted_rows:
                row = sorted(row)  
                to_remove = []
                current = None
                for l in row:
                    if current == None:
                        current = l
                    elif l[0] == current[0] + 1:
                        # Must be adjacent, add to the to remove list.
                        to_remove.append(current)
                        current = l
                    else:
                        # Not at the start, and not adjacent, so is a break in the side.
                        current = l
                area_sides += len([x for x in row if x not in to_remove])
    
    new_total += area_sides * area_areas[area_counter]
    area_counter += 1
    
print(new_total)

887932
